In [ ]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using Statistics
using Measures
using StatsBase

using CairoMakie
CairoMakie.activate!(type="png")
set_theme!(resolution=(400, 400))
CairoMakie.inline!(true)

using GraphMakie: graphplot
using GraphMakie: graphplot!

using CairoMakie.Colors
using GeometryBasics: Point
using FFMPEG

#### plotting utility

In [ ]:
function plot_step(sub_g,meta_g,n_side)
    ns = 1
    rs = 10
    edgecolors = [:blue for i in 1:ne(sub_g)]
    nodecolors = [:black for i in 1:nv(sub_g)]
    nodesizes = [ns for i in 1:nv(sub_g)]

    a = 0
    b = 1
    # number of nodes
    n_v = n_side^2
    x = flat_square_2d_grid(n_v, a, b)

    # grid spacing
    grid_spacing = (b-a)/(n_side - 1) 
    
    # color the roots
    roots = get_prop(sub_g, :roots)
    for v in roots
        nodecolors[v] = :red;
        nodesizes[v] = 15;
    end

    # color the starting nodes
    marked_nodes = get_prop(sub_g, :marked_node)
    for v in marked_nodes
        nodecolors[v] = :blue;
        nodesizes[v] = 15;
    end

    # construct graph with only the cycles and all remaining nodes are isolated
    cycles = get_prop(sub_g, :cycle_nodes)
    cycle_g = MetaGraph(n_v)

    for c in cycles 
        for u in 1:n_v
            for v in 1:n_v
                if u < v
                    edge_exits = (norm(x[u,:]-x[v,:],1) < grid_spacing + 1e-10)
                    u_in_c = (indexin(u, c)[1] !== nothing)
                    v_in_c = (indexin(v, c)[1] !== nothing)

                    if u_in_c && v_in_c && edge_exits && has_edge(sub_g, Edge(u,v))
                        add_edge!(cycle_g, u, v)
                    end
                end
            end
        end
    end

    cyclenodecolors = [:black for i in 1:nv(cycle_g)]
    cycleedgecolors = [:red for i in 1:ne(cycle_g)]
    cyclenodesizes = [ns for i in 1:nv(cycle_g)]


    f, ax, p = graphplot(meta_g; layout=mylayout)    
    graphplot!(sub_g;
        layout = mylayout,
        edge_color = edgecolors,
        node_color = nodecolors,
        node_size = nodesizes,
        edge_width = 3.5)
    graphplot!(cycle_g;
        layout=mylayout,
        node_color=cyclenodecolors,
        edge_color=cycleedgecolors,
        node_size = cyclenodesizes,
        edge_width = 4)

    hidedecorations!(ax); hidespines!(ax)
    ax.aspect = DataAspect()

    return f, ax, p
end

#### Connection graph construction

In [ ]:
spanning_graph = "MTSF"
# several options "ST", "SF", "CRSF", "MTSF"

##  default parameters
# number of nodes on the side
n_side = 7

# interval [a,b]^2
a = 0
b = 1

## initializations

# for sampling
absorbing_node = false
ust = false # for spanning trees

# proba to have an outlier edge
η = 0.
# uniform noise on edges
noise = 0.
# regularization parameter
q = 0.

if spanning_graph == "ST"
    absorbing_node = true
    ust = true
    η = 0.
    noise = 0.
elseif spanning_graph == "SF"
    q = 0.1
    η = 0.
    noise = 0.
elseif spanning_graph == "CRSF"
    η = 0.2
    noise = 0.2
elseif spanning_graph == "MTSF"
    q = 0.05
    η = 0.2
    noise = 0.
end

# edges weights?
weighted = false

# random generator
rng = Random.default_rng()

# number of nodes
n_v = n_side^2

# grid spacing
grid_spacing = (b-a)/(n_side - 1) 
# grid coordinates
x = flat_square_2d_grid(n_v, a, b)
# for plotting
function mylayout(g::AbstractGraph)
    # create iterator going over x-coord and y-coord 
    return Point.(zip(x[:, 1], x[:, 2]))
end

# planted ranking score
planted_score = randperm(rng,n_v);

meta_g = MetaGraph(n_v)

for u in 1:n_v
    for v in 1:n_v
        if u < v
            edge_exits = (norm(x[u,:]-x[v,:],1) < grid_spacing + 1e-10)
            if edge_exits 
                h_u = planted_score[u]
                h_v = planted_score[v]
                θ = (h_u - h_v) * π / (n_v - 1)
                if (rand(rng) < η)
                    θ = rand(rng, (-n_v + 1):(n_v - 1)) * π / (n_v - 1)
                else
                    θ *= 1.0 + noise * 2 * (rand(rng) - 0.5)
                end
                add_edge!(meta_g, u, v, :angle, θ)
            end
        end
    end
end



#### MTSF step by step

In [ ]:
# full graph
g = meta_g
# empty mtsf
mtsf = MetaGraph(nv(g))
# temporary graph to display alg progress
mtsf_temp = MetaGraph(nv(g))

nv_mtsf = 0
weight = 1.0
roots = []
nodes_in_cycles = []
set_prop!(mtsf_temp, :roots, roots)
set_prop!(mtsf_temp, :cycle_nodes, nodes_in_cycles)

reverse_order_branches = Vector[]

list_of_graphs = []
temp_g = MetaGraph(nv(g))

# Initialize the random walk
walk = []
unvisited = Set(vertices(g))

#fix a root if necessary
if absorbing_node
    ab_node = rand(rng, unvisited)
    set_prop!(mtsf_temp, :roots, ab_node)

    push!(roots, ab_node)
    setdiff!(unvisited, ab_node)
    nv_mtsf += 1
end

# Start the random walk
n0 = rand(rng, unvisited)
# add n0 to walk
push!(walk, n0)
# mark n0 as visited
setdiff!(unvisited, n0)

set_prop!(mtsf_temp, :marked_node, n0)
print("stored starting node \n")
push!(list_of_graphs,copy(mtsf_temp))

while nv_mtsf < nv(g)

    # check if n0 is a root
    n0_is_root = false
    if q > 1e-10 # namely if q large enough
        #n0_is_root = rand(rng) < q / (q + degree(g, n0))
        n0_is_root = step_to_root(rng, g, n0, q, weighted)
    end

    if n0_is_root # if n0 is indeed a root
        # record the rooted branch in mtsf
        push!(roots, n0)
        set_prop!(mtsf_temp, :roots, roots)

        add_edges_from!(mtsf, consecutive_pairs(walk))
        nv_mtsf += length(walk)
        # mark branch as visisted
        setdiff!(unvisited, walk)

        # record branch w/o  root
        push!(reverse_order_branches, walk[1:(end - 1)])
        # restart with a new n0 uniformly among the unvisited nodes
        n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
        continue
    end

    # walk to a neighbour wrt degree distribution
    n1 = rand_step(rng, g, n0, weighted)
    # add neighbour to walk
    push!(walk, n1)

    # mark nb as current point
    set_prop!(mtsf_temp, :marked_node, n1)
    add_edges_from!(mtsf_temp, partition([n0 n1], 2, 1))
    push!(list_of_graphs,copy(mtsf_temp))
    print("stored edge \n")
    

    if n1 in unvisited
        setdiff!(unvisited, n1)
        n0 = n1  # if n1 unvisited continue the walk

    
    # otherwise n1 is already visited    
    elseif (degree(mtsf, n1) > 0 || n1 in roots) || (n1 in roots && ust)
        print("branch merged ")
        add_edges_from!(mtsf, consecutive_pairs(walk))
        nv_mtsf += length(walk) - 1
        setdiff!(unvisited, walk)

        push!(reverse_order_branches, walk[1:(end - 1)])

        n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
        print("stored current node \n")

        set_prop!(mtsf_temp, :marked_node, n0)
        print("and restart walk \n")

    else  # if n1 in walk: identify unique cycle/loop in walk with knot n1

        print("cycle formed: ")
        idx_n1 = findfirst(x -> x == n1, walk)
        cycle_nodes = @view walk[idx_n1:end]
        keep = false # by default pop cycle
        alpha = 0
        if !ust # if we are not in the spanning tree case, we can try to accept the cycle
            keep, alpha = keep_cycle(rng, g, consecutive_pairs(cycle_nodes))
        end

        if keep # keep cycle
            print(" accepted ")
            weight *= max(alpha, 1)
            add_edges_from!(mtsf, consecutive_pairs(walk))
            nv_mtsf += length(walk) - 1 # since walk contains twice the knot
            setdiff!(unvisited, walk)

            # record cycle nodes: remove starting node so that it appears only once
            push!(nodes_in_cycles, cycle_nodes[2:end])
            # record branch without the knot
            push!(reverse_order_branches, walk[1:(idx_n1 - 1)])
            set_prop!(mtsf_temp, :cycle_nodes, nodes_in_cycles)

            print("and restart walk\n")
            n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
            set_prop!(mtsf_temp, :marked_node, n0)
            print("stored current node \n")
            push!(list_of_graphs,copy(mtsf_temp))



        else  # pop cycle but keep the knot
            print(" popped ")
            union!(unvisited, cycle_nodes)
            setdiff!(unvisited, n1)  # remove n1 which was part of cycle_nodes
            resize!(walk, idx_n1)
            # restart from the knot
            n0 = n1
            set_prop!(mtsf_temp, :marked_node, n0)
            print("stored current node \n")
            print("and restart walk\n")

            rem_edges_from!(mtsf_temp, consecutive_pairs(cycle_nodes))
            push!(list_of_graphs,copy(mtsf_temp))
        end
    end
end

set_prop!(mtsf_temp, :marked_node, [])

push!(list_of_graphs,copy(mtsf_temp))


In [ ]:
n_frames = length(list_of_graphs)

for i = 1:n_frames
    f,ax,p = plot_step(list_of_graphs[i],meta_g,n_side)
    display(f)
    name = spanning_graph * "_" * string(i)
    save("figures/"*name*".png", f, pt_per_unit = 0.5)
    sleep(0.1)
end

# add a few extra fixed frames at the end of the gif
n_extra = 5

for i = (n_frames + 1) : (n_frames + n_extra)
    f,ax,p = plot_step(list_of_graphs[end],meta_g,n_side)
    display(f)
    name = spanning_graph * "_" * string(i)
    save("figures/"*name*".png", f, pt_per_unit = 0.5)
    sleep(0.1)
end

In [ ]:
using FFMPEG
imagesdirectory = "/Users/mfanuel/Documents/Code/juliaTemplate/MagneticLaplacianSparsifier/notebooks/figures/"
framerate = 10#30
gifname = "/Users/mfanuel/Documents/Code/juliaTemplate/MagneticLaplacianSparsifier/notebooks/gifs/"
gifname = gifname * spanning_graph * "_grid.gif"
gifpalette = "/Users/mfanuel/Documents/Code/juliaTemplate/MagneticLaplacianSparsifier/notebooks/gifs/palette.png"

imagesdirectory = imagesdirectory * spanning_graph 
#rm(gifname)
#rm(gifpalette)

FFMPEG.ffmpeg_exe(`-framerate $(framerate) -f image2 -i $(imagesdirectory)_%1d.png -vf palettegen -y $(gifpalette)`)

FFMPEG.ffmpeg_exe(`-framerate $(framerate) -f image2 -i $(imagesdirectory)_%1d.png -i $(gifpalette) -lavfi paletteuse -y $(gifname)`)


In [ ]:
# old things

# gifs yellowish
#FFMPEG.ffmpeg_exe(`-framerate $(framerate) -f image2 -i $(imagesdirectory)simple_%1d.png -y $(gifname)`)
# for video work fine
#videoname = "/Users/mfanuel/Documents/Code/juliaTemplate/MagneticLaplacianSparsifier/notebooks/figures/gifs/output.mov"
#FFMPEG.ffmpeg_exe(`-framerate $(framerate) -f image2 -i $(imagesdirectory)simple_%1d.png -vf "scale=1080:1080" -c:v libx264 -pix_fmt yuv420p -y $(videoname)`)